<a href="https://colab.research.google.com/github/lmcanavals/compilers/blob/main/0201_introlex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%file commandline.cpp
#include <iostream>
#include <fstream>

using namespace std;

int main(int argc, char** argv) {
    for (int i = 0; i < argc; ++i) {
        cout << "Argumento " << i << ": " <<  argv[i] << endl;
    }
    return 0;
}

Overwriting scanner1.cpp


Compilamos el programa con gcc, el compilador para c++ se llama g++

In [ ]:
!g++ commandline.cpp

En linux listamos los archivos en la carpeta actual con ls -l, nótese que se ha creado el archivo a.out (en caso de linux o a.exe en caso de windows)

In [ ]:
!ls -l

total 20
-rwxr-xr-x 1 root root 8976 Sep  2 20:45 a.out
drwxr-xr-x 1 root root 4096 Aug 31 13:47 sample_data
-rw-r--r-- 1 root root  218 Sep  2 20:45 scanner1.cpp


Ejecutamos en línea de comandos, colocando argumentos delante del programa, los cuales seran leidos en argv.

In [ ]:
!./a.out hola estos son los parametros en linea de comandos, es decir cada cosa que pongamos delante "incluso puedo poner varias cosas con comillas"

Argumento 0: ./a.out
Argumento 1: hola
Argumento 2: estos
Argumento 3: son
Argumento 4: los
Argumento 5: parametros
Argumento 6: en
Argumento 7: linea
Argumento 8: de
Argumento 9: comandos,
Argumento 10: es
Argumento 11: decir
Argumento 12: cada
Argumento 13: cosa
Argumento 14: que
Argumento 15: pongamos
Argumento 16: delante
Argumento 17: incluso puedo poner varias cosas con comillas


In [ ]:
!./a.out

Argumento 0: ./a.out


In [ ]:
%%file scanner1.cpp
#include <iostream>
#include <fstream>

using namespace std;

int main(int argc, char** argv) {
    if (argc < 2) {
        cerr << "Usage: " << argv[0] << " programafuente\n";
        return -1;
    }

    ifstream f(argv[1]);

    char c;
    while (f.get(c)) {
        cout << c << endl;
    }
    
    f.close();
    return 0;
}

Overwriting scanner1.cpp


In [ ]:
!g++ scanner1.cpp && ./a.out scanner1.cpp

## Flex

In [ ]:
!sudo apt install flex

In [31]:
%%file simple1.l
%%
username    printf( "%s", getlogin() );



Writing simple1.l


In [32]:
!flex simple1.l

In [ ]:
!ls -l

In [35]:
!gcc lex.yy.c -ll

Primer ejemplo de Flex

In [38]:
%%file test
hola, username
saludos

Writing test


In [39]:
!./a.out < test

hola, (null)
saludos


In [40]:
!whoami

root


Segundo ejemplo de flex

In [41]:
%%file simple2.l
        int num_lines = 0, num_chars = 0;

%%
\n      ++num_lines; ++num_chars;
.       ++num_chars;

%%
int main() {
    yylex();
    printf( "# of lines = %d, # of chars = %d\n", num_lines, num_chars );

    return 0;
}

Writing simple2.l


In [42]:
!flex simple2.l && gcc lex.yy.c -ll && ./a.out < simple2.l

# of lines = 13, # of chars = 222


Un ejemplo más elaborado

In [43]:
%%file simplepascal.l
/* scanner for a toy Pascal-like language */

%{
/* need this for the call to atof() below */
#include <math.h>
%}

DIGIT    [0-9]
ID       [a-z][a-z0-9]*

%%

{DIGIT}+    {
            printf("An integer: %s (%d)\n", yytext, atoi(yytext));
            }

{DIGIT}+"."{DIGIT}*        {
            printf("A float: %s (%g)\n", yytext, atof(yytext));
            }

if|then|begin|end|procedure|function        {
            printf("A keyword: %s\n", yytext);
            }

{ID}        printf("An identifier: %s\n", yytext);

"+"|"-"|"*"|"/"   printf( "An operator: %s\n", yytext );

"{"[^}\n]*"}"     /* eat up one-line comments */

[ \t\n]+          /* eat up whitespace */

.           printf( "Unrecognized character: %s\n", yytext );

%%

int main(int argc, char** argv) {
    ++argv, --argc;  /* skip over program name */
    if ( argc > 0 )
            yyin = fopen( argv[0], "r" );
    else
            yyin = stdin;

    yylex();

    return 0;
}

Writing simplepascal.l


In [44]:
%%file hola.pas
program HelloWorld;
uses crt;

{ Here the main program block starts }
begin
   writeln('Hello, World!');
   readkey;
end. 

Writing hola.pas


In [45]:
!flex simplepascal.l && gcc lex.yy.c -ll && ./a.out < hola.pas

An identifier: program
Unrecognized character: H
An identifier: ello
Unrecognized character: W
An identifier: orld
Unrecognized character: ;
An identifier: uses
An identifier: crt
Unrecognized character: ;
A keyword: begin
An identifier: writeln
Unrecognized character: (
Unrecognized character: '
Unrecognized character: H
An identifier: ello
Unrecognized character: ,
Unrecognized character: W
An identifier: orld
Unrecognized character: !
Unrecognized character: '
Unrecognized character: )
Unrecognized character: ;
An identifier: readkey
Unrecognized character: ;
A keyword: end
Unrecognized character: .
